In [29]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import sys
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

### Read and view data tables

In [4]:
"""
Filenames:
calendar.csv
sales_train_evaluation.csv
sales_train_validation.csv
sales_submission.csv
sell_prices.csv
"""

'\nFilenames:\ncalendar.csv\nsales_train_evaluation.csv\nsales_train_validation.csv\nsales_submission.csv\nsell_prices.csv\n'

In [27]:
folder = 'm5-forecasting-accuracy/'
df_calendar = pd.read_csv(folder + 'calendar.csv')
df_sales_train_validation = pd.read_csv(folder + 'sales_train_validation.csv')
df_sell_prices = pd.read_csv(folder + 'sell_prices.csv')

In [6]:
"""
Comments:
- Use table to link dates to events
- Ignore 'snap_' columns
"""

print(df_calendar.head(5))
print(df_calendar.describe())
print(df_calendar.event_name_1.unique())
print(df_calendar.event_type_1.unique())
print(df_calendar.event_name_2.unique())
print(df_calendar.event_type_2.unique())
print(df_calendar.wm_yr_wk.unique())

         date  wm_yr_wk    weekday  wday  month  year    d event_name_1  \
0  2011-01-29     11101   Saturday     1      1  2011  d_1          NaN   
1  2011-01-30     11101     Sunday     2      1  2011  d_2          NaN   
2  2011-01-31     11101     Monday     3      1  2011  d_3          NaN   
3  2011-02-01     11101    Tuesday     4      2  2011  d_4          NaN   
4  2011-02-02     11101  Wednesday     5      2  2011  d_5          NaN   

  event_type_1 event_name_2 event_type_2  snap_CA  snap_TX  snap_WI  
0          NaN          NaN          NaN        0        0        0  
1          NaN          NaN          NaN        0        0        0  
2          NaN          NaN          NaN        0        0        0  
3          NaN          NaN          NaN        1        1        0  
4          NaN          NaN          NaN        1        0        1  
           wm_yr_wk         wday        month         year      snap_CA  \
count   1969.000000  1969.000000  1969.000000  1969.00

In [7]:
print(df_sales_train_validation.head(5))
print(df_sales_train_validation.dept_id.unique())
print(df_sales_train_validation.item_id.unique())

                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id  d_1  d_2  d_3  d_4  ...  d_1904  d_1905  d_1906  d_1907  d_1908  \
0       CA    0    0    0    0  ...       1       3       0       1       1   
1       CA    0    0    0    0  ...       0       0       0       0       0   
2       CA    0    0    0    0  ...       2       1       2       1       1   
3       CA    0    0    0    0  ...       1       0       5       4       1   
4       CA    0    0    0    0  ...       2       1       1       0       1   

   d_1909  d_1910  d_1911  d_1912  d_1913  


In [8]:
print(df_sell_prices.head(5))
print(df_sell_prices.describe())
print(sys.getsizeof(df_sell_prices))

  store_id        item_id  wm_yr_wk  sell_price
0     CA_1  HOBBIES_1_001     11325        9.58
1     CA_1  HOBBIES_1_001     11326        9.58
2     CA_1  HOBBIES_1_001     11327        8.26
3     CA_1  HOBBIES_1_001     11328        8.26
4     CA_1  HOBBIES_1_001     11329        8.26
           wm_yr_wk    sell_price
count  6.841121e+06  6.841121e+06
mean   1.138294e+04  4.410952e+00
std    1.486100e+02  3.408814e+00
min    1.110100e+04  1.000000e-02
25%    1.124700e+04  2.180000e+00
50%    1.141100e+04  3.470000e+00
75%    1.151700e+04  5.840000e+00
max    1.162100e+04  1.073200e+02
1004032223


### Combine into one table

In [9]:
df_intermediate = df_calendar.merge(df_sell_prices, how='left', on='wm_yr_wk')

In [10]:
print(df_intermediate.head(10))
print(sys.getsizeof(df_intermediate))

         date  wm_yr_wk   weekday  wday  month  year    d event_name_1  \
0  2011-01-29     11101  Saturday     1      1  2011  d_1          NaN   
1  2011-01-29     11101  Saturday     1      1  2011  d_1          NaN   
2  2011-01-29     11101  Saturday     1      1  2011  d_1          NaN   
3  2011-01-29     11101  Saturday     1      1  2011  d_1          NaN   
4  2011-01-29     11101  Saturday     1      1  2011  d_1          NaN   
5  2011-01-29     11101  Saturday     1      1  2011  d_1          NaN   
6  2011-01-29     11101  Saturday     1      1  2011  d_1          NaN   
7  2011-01-29     11101  Saturday     1      1  2011  d_1          NaN   
8  2011-01-29     11101  Saturday     1      1  2011  d_1          NaN   
9  2011-01-29     11101  Saturday     1      1  2011  d_1          NaN   

  event_type_1 event_name_2 event_type_2  snap_CA  snap_TX  snap_WI store_id  \
0          NaN          NaN          NaN        0        0        0     CA_1   
1          NaN          N

In [11]:
days = ['d_' + str(i) for i in range(1, 1914)]
df_sales_train_validation_pivot = pd.melt(df_sales_train_validation, id_vars=['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], value_vars=days, var_name='d', value_name='unit_sales')
print(df_sales_train_validation_pivot.head(10))
print(df_sales_train_validation_pivot.size)

         item_id    dept_id   cat_id store_id state_id    d  unit_sales
0  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1       CA  d_1           0
1  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1       CA  d_1           0
2  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1       CA  d_1           0
3  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1       CA  d_1           0
4  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1       CA  d_1           0
5  HOBBIES_1_006  HOBBIES_1  HOBBIES     CA_1       CA  d_1           0
6  HOBBIES_1_007  HOBBIES_1  HOBBIES     CA_1       CA  d_1           0
7  HOBBIES_1_008  HOBBIES_1  HOBBIES     CA_1       CA  d_1          12
8  HOBBIES_1_009  HOBBIES_1  HOBBIES     CA_1       CA  d_1           2
9  HOBBIES_1_010  HOBBIES_1  HOBBIES     CA_1       CA  d_1           0
408291590


In [22]:
df_stvp_item = df_sales_train_validation_pivot[df_sales_train_validation_pivot.item_id == 'FOODS_3_069']
df_intermediate_item = df_intermediate[df_intermediate.item_id == 'FOODS_3_069']

In [23]:
cols_to_use = df_intermediate_item.columns.difference(df_stvp_item.columns).tolist()
cols_to_use = cols_to_use + ['item_id', 'd', 'store_id']
df_merged = df_stvp_item.merge(df_intermediate_item[cols_to_use], how='inner', on=['item_id', 'd', 'store_id'])
df_merged.to_csv('df_merged.csv')
print(df_merged.head(10))
print(df_merged.columns)

       item_id  dept_id cat_id store_id state_id    d  unit_sales        date  \
0  FOODS_3_069  FOODS_3  FOODS     CA_2       CA  d_1           0  2011-01-29   
1  FOODS_3_069  FOODS_3  FOODS     TX_2       TX  d_1           5  2011-01-29   
2  FOODS_3_069  FOODS_3  FOODS     WI_1       WI  d_1           0  2011-01-29   
3  FOODS_3_069  FOODS_3  FOODS     WI_2       WI  d_1          13  2011-01-29   
4  FOODS_3_069  FOODS_3  FOODS     WI_3       WI  d_1           3  2011-01-29   
5  FOODS_3_069  FOODS_3  FOODS     CA_2       CA  d_2           1  2011-01-30   
6  FOODS_3_069  FOODS_3  FOODS     TX_2       TX  d_2           3  2011-01-30   
7  FOODS_3_069  FOODS_3  FOODS     WI_1       WI  d_2           0  2011-01-30   
8  FOODS_3_069  FOODS_3  FOODS     WI_2       WI  d_2           1  2011-01-30   
9  FOODS_3_069  FOODS_3  FOODS     WI_3       WI  d_2           0  2011-01-30   

  event_name_1 event_name_2 event_type_1 event_type_2  month  sell_price  \
0          NaN          NaN     

### Analyze single store and single item sales

In [34]:
df_merged_store = df_merged[df_merged.store_id == 'TX_1']
df_merged_store.reset_index(drop=True, inplace=True)
df_merged_store['unit_sales_avg'] = df_merged_store['unit_sales'].rolling(7).mean()
print(df_merged_store.head(5))

fig = px.line(df_merged_store, x='d', y='unit_sales_avg', color='store_id')
fig.show()

       item_id  dept_id cat_id store_id state_id     d  unit_sales  \
0  FOODS_3_069  FOODS_3  FOODS     TX_1       TX  d_50           0   
1  FOODS_3_069  FOODS_3  FOODS     TX_1       TX  d_51           3   
2  FOODS_3_069  FOODS_3  FOODS     TX_1       TX  d_52           0   
3  FOODS_3_069  FOODS_3  FOODS     TX_1       TX  d_53           0   
4  FOODS_3_069  FOODS_3  FOODS     TX_1       TX  d_54           0   

         date event_name_1 event_name_2 event_type_1 event_type_2  month  \
0  2011-03-19          NaN          NaN          NaN          NaN      3   
1  2011-03-20    Purim End          NaN    Religious          NaN      3   
2  2011-03-21          NaN          NaN          NaN          NaN      3   
3  2011-03-22          NaN          NaN          NaN          NaN      3   
4  2011-03-23          NaN          NaN          NaN          NaN      3   

   sell_price  snap_CA  snap_TX  snap_WI  wday    weekday  wm_yr_wk  year  \
0        2.97        0        0        0     

/home/ben/anaconda3/envs/main37/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [42]:
print(df_merged_store.head(10))
date = pd.PeriodIndex(df_merged_store['date'], freq="D")
df_merged_store['date'] = date
df_merged_store.set_index('date', inplace=True)

fourier = CalendarFourier(freq='A', order=4)
dp = DeterministicProcess(index=df_merged_store.index,
                                    constant=True,
                                    order=1,
                                    seasonal=True,
                                    additional_terms=[fourier],
                                    drop=True)

       item_id  dept_id cat_id store_id state_id     d  unit_sales  \
0  FOODS_3_069  FOODS_3  FOODS     TX_1       TX  d_50           0   
1  FOODS_3_069  FOODS_3  FOODS     TX_1       TX  d_51           3   
2  FOODS_3_069  FOODS_3  FOODS     TX_1       TX  d_52           0   
3  FOODS_3_069  FOODS_3  FOODS     TX_1       TX  d_53           0   
4  FOODS_3_069  FOODS_3  FOODS     TX_1       TX  d_54           0   
5  FOODS_3_069  FOODS_3  FOODS     TX_1       TX  d_55           0   
6  FOODS_3_069  FOODS_3  FOODS     TX_1       TX  d_56           0   
7  FOODS_3_069  FOODS_3  FOODS     TX_1       TX  d_57           5   
8  FOODS_3_069  FOODS_3  FOODS     TX_1       TX  d_58           1   
9  FOODS_3_069  FOODS_3  FOODS     TX_1       TX  d_59           0   

         date event_name_1 event_name_2 event_type_1 event_type_2  month  \
0  2011-03-19          NaN          NaN          NaN          NaN      3   
1  2011-03-20    Purim End          NaN    Religious          NaN      3   
2

/home/ben/anaconda3/envs/main37/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Find foods that sell more on holidays

In [15]:
df_stvp_foods = df_sales_train_validation_pivot[df_sales_train_validation_pivot.cat_id == 'FOODS']
df_stvp_foods = df_stvp_foods.groupby(by=['d', 'item_id'], axis=0, as_index=False).mean()
print(df_stvp_foods.size)
print(df_stvp_foods.head(5))

8246943
     d      item_id  unit_sales
0  d_1  FOODS_1_001         0.6
1  d_1  FOODS_1_002         0.4
2  d_1  FOODS_1_003         1.4
3  d_1  FOODS_1_004         0.0
4  d_1  FOODS_1_005         3.4


In [16]:
df_merged = df_stvp_foods.merge(df_calendar, how='left', on='d')
print(df_merged.head(10))
print(df_merged.columns)

     d      item_id  unit_sales        date  wm_yr_wk   weekday  wday  month  \
0  d_1  FOODS_1_001         0.6  2011-01-29     11101  Saturday     1      1   
1  d_1  FOODS_1_002         0.4  2011-01-29     11101  Saturday     1      1   
2  d_1  FOODS_1_003         1.4  2011-01-29     11101  Saturday     1      1   
3  d_1  FOODS_1_004         0.0  2011-01-29     11101  Saturday     1      1   
4  d_1  FOODS_1_005         3.4  2011-01-29     11101  Saturday     1      1   
5  d_1  FOODS_1_006         1.6  2011-01-29     11101  Saturday     1      1   
6  d_1  FOODS_1_008         0.0  2011-01-29     11101  Saturday     1      1   
7  d_1  FOODS_1_009         0.0  2011-01-29     11101  Saturday     1      1   
8  d_1  FOODS_1_010         0.0  2011-01-29     11101  Saturday     1      1   
9  d_1  FOODS_1_011         0.7  2011-01-29     11101  Saturday     1      1   

   year event_name_1 event_type_1 event_name_2 event_type_2  snap_CA  snap_TX  \
0  2011          NaN          NaN     

In [17]:
df_mean_sales_by_event = df_merged.groupby(by=['item_id', 'event_name_1'], axis=0, dropna=False, as_index=False).mean()
df_mean_sales_by_event.to_csv('df_mean_sales_by_event.csv')
print(df_mean_sales_by_event.head(5))

       item_id   event_name_1  unit_sales  wm_yr_wk  wday  month    year  \
0  FOODS_1_001   Chanukah End       0.500  11346.60   4.2   12.0  2013.0   
1  FOODS_1_001      Christmas       0.000  11347.60   4.8   12.0  2013.0   
2  FOODS_1_001  Cinco De Mayo       0.725  11314.25   3.5    5.0  2013.0   
3  FOODS_1_001    ColumbusDay       0.600  11337.20   3.0   10.0  2013.0   
4  FOODS_1_001         Easter       0.520  11410.40   2.0    3.6  2014.0   

   snap_CA  snap_TX  snap_WI  
0      0.2      0.2      0.4  
1      0.0      0.0      0.0  
2      1.0      1.0      1.0  
3      0.4      0.4      0.6  
4      0.4      0.2      0.4  


In [18]:
df_mean_sales_by_event_pivot = df_mean_sales_by_event.pivot(index='item_id', columns='event_name_1', values='unit_sales')
df_mean_sales_by_event_pivot.rename(columns={df_mean_sales_by_event_pivot.columns[0]: 'None'}, inplace=True)
print(df_mean_sales_by_event_pivot.head(5))

event_name_1      None  Chanukah End  Christmas  Cinco De Mayo  ColumbusDay  \
item_id                                                                       
FOODS_1_001   0.644116          0.50        0.0          0.725         0.60   
FOODS_1_002   0.386754          0.40        0.0          0.150         0.24   
FOODS_1_003   0.707447          0.78        0.0          0.975         0.58   
FOODS_1_004   6.736896          7.26        0.0          8.975         4.86   
FOODS_1_005   1.188459          2.48        0.0          0.775         2.06   

event_name_1  Easter  Eid al-Fitr  EidAlAdha  Father's day  Halloween  ...  \
item_id                                                                ...   
FOODS_1_001     0.52         0.50       0.48         0.925       0.54  ...   
FOODS_1_002     0.46         0.38       0.52         0.325       0.34  ...   
FOODS_1_003     0.64         0.50       0.98         0.850       0.70  ...   
FOODS_1_004    12.80         6.82       5.98         8.0

In [19]:
pd.set_option("display.max_columns", None)
df_sales_ratio_by_event = df_mean_sales_by_event_pivot.div(df_mean_sales_by_event_pivot['None'], axis=0)
df_sales_ratio_by_event.sort_values('Thanksgiving', axis=0, ascending=False, inplace=True)
print(df_sales_ratio_by_event.head(5))

event_name_1  None  Chanukah End  Christmas  Cinco De Mayo  ColumbusDay  \
item_id                                                                   
FOODS_3_069    1.0      4.019466        0.0       0.497060     0.483626   
FOODS_2_255    1.0      1.610159        0.0       0.343631     0.746171   
FOODS_2_096    1.0      2.143255        0.0       0.765448     1.224717   
FOODS_3_015    1.0      1.701533        0.0       1.003262     0.674192   
FOODS_2_066    1.0      1.446960        0.0       0.982989     1.163859   

event_name_1    Easter  Eid al-Fitr  EidAlAdha  Father's day  Halloween  \
item_id                                                                   
FOODS_3_069   1.235932     0.440637   0.752306      0.591098   0.881273   
FOODS_2_255   0.719990     0.405812   0.890169      0.539992   0.693808   
FOODS_2_096   5.817406     0.918538   0.306179      0.382724   0.612359   
FOODS_3_015   1.348385     0.722349   0.866819      1.123654   0.850767   
FOODS_2_066   1.730061  

In [20]:
df_sales_ratio_by_event.sort_values('ValentinesDay', axis=0, ascending=False, inplace=True)
print(df_sales_ratio_by_event.head(5))

event_name_1  None  Chanukah End  Christmas  Cinco De Mayo  ColumbusDay  \
item_id                                                                   
FOODS_1_180    1.0      2.373848        0.0       1.015836     0.620195   
FOODS_1_055    1.0      2.453128        0.0       0.367635     0.802113   
FOODS_3_796    1.0      2.182382        0.0       0.000000     1.018445   
FOODS_1_037    1.0      1.362476        0.0       1.486829     1.470609   
FOODS_1_123    1.0      1.796171        0.0       0.904322     1.197447   

event_name_1    Easter  Eid al-Fitr  EidAlAdha  Father's day  Halloween  \
item_id                                                                   
FOODS_1_180   0.919599     0.834055   1.090687      1.202964   0.662967   
FOODS_1_055   0.648375     0.802113   0.895693      1.554094   1.029378   
FOODS_3_796   0.872953     0.000000   0.436476      0.000000   1.745906   
FOODS_1_037   2.487060     0.670425   0.389279      0.486599   0.000000   
FOODS_1_123   1.721330  